In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from pandas.api.types import is_numeric_dtype
from pprint import pprint, pformat
import zipfile
from pathlib import Path


import joblib

from my_lib import *
from my_config import *

# 1. Загрузить датасет

In [2]:
# https://www.kaggle.com/competitions/regression-with-an-insurance-dataset-clone/data
# https://www.kaggle.com/datasets/schran/insurance-premium-prediction
with zipfile.ZipFile('dataset/train.zip', 'r') as zip_ref:
    #print(zip_ref.__dir__())
    dataset_df = pd.read_csv(zip_ref.open('train.csv'), index_col="id")

# 2. EDA датасета

In [3]:
display(dataset_df.sample(7))
display(dataset_df.info())

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
id,,,,,,,,,,,,,,,,,,,,
121294,37.0,Male,15107.0,Married,0.0,Bachelor's,NaN,26.396739,Urban,Comprehensive,NaN,9.0,638.0,5.0,2021-12-16 15:21:39.286113,Average,No,Rarely,Condo,3138.0
573020,19.0,Male,NaN,Divorced,4.0,PhD,Self-Employed,19.791179,Urban,Premium,0.0,5.0,338.0,2.0,2022-03-03 15:21:39.086123,Good,Yes,Monthly,House,991.0
343226,42.0,Male,56780.0,Divorced,1.0,PhD,NaN,34.945284,Rural,Basic,1.0,11.0,736.0,3.0,2020-06-16 15:21:39.103944,Good,Yes,Weekly,Condo,160.0
599675,53.0,Male,38983.0,Divorced,4.0,PhD,Self-Employed,29.924650,Urban,Basic,1.0,4.0,541.0,8.0,2019-12-27 15:21:39.177724,Poor,Yes,Rarely,Condo,148.0
854147,32.0,Male,25148.0,Married,3.0,PhD,Self-Employed,33.386773,Rural,Comprehensive,NaN,16.0,767.0,9.0,2022-10-09 15:21:39.121169,Average,Yes,Weekly,House,293.0
375027,31.0,Female,14014.0,Divorced,0.0,Bachelor's,NaN,29.053874,Urban,Comprehensive,NaN,8.0,339.0,5.0,2023-09-30 15:21:39.151732,Poor,Yes,Weekly,Apartment,503.0
155334,29.0,Female,24682.0,Single,0.0,Bachelor's,NaN,17.918090,Suburban,Premium,NaN,19.0,763.0,2.0,2023-07-09 15:21:39.237118,Poor,Yes,Rarely,House,942.0


<class 'pandas.core.frame.DataFrame'>
Index: 1200000 entries, 0 to 1199999
Data columns (total 20 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   Age                   1181295 non-null  float64
 1   Gender                1200000 non-null  object 
 2   Annual Income         1155051 non-null  float64
 3   Marital Status        1181471 non-null  object 
 4   Number of Dependents  1090328 non-null  float64
 5   Education Level       1200000 non-null  object 
 6   Occupation            841925 non-null   object 
 7   Health Score          1125924 non-null  float64
 8   Location              1200000 non-null  object 
 9   Policy Type           1200000 non-null  object 
 10  Previous Claims       835971 non-null   float64
 11  Vehicle Age           1199994 non-null  float64
 12  Credit Score          1062118 non-null  float64
 13  Insurance Duration    1199999 non-null  float64
 14  Policy Start Date     1200000 non-null 

None

In [4]:
# выделим числовые и категориальные переменные
params = {}
params["target_column"] = "Premium Amount"
# columns_X - переменные датасета
params["columns_X"] = dataset_df.columns.to_list()
params["columns_X"].remove(params["target_column"])
params["num_columns"] = []
params["cat_columns"] = []
for col in params["columns_X"]:
    if dataset_df[col].nunique() <= 2 or not is_numeric_dtype(dataset_df[col]):
        params["cat_columns"].append(col)
    else:
        params["num_columns"].append(col)
print(f'target_columns={params["target_column"]}')        
print(f'columns_X={params["columns_X"]}')
print(f'cat_columns={pformat(params["cat_columns"])}')
print(f'num_columns={pformat(params["num_columns"])}')

target_columns=Premium Amount
columns_X=['Age', 'Gender', 'Annual Income', 'Marital Status', 'Number of Dependents', 'Education Level', 'Occupation', 'Health Score', 'Location', 'Policy Type', 'Previous Claims', 'Vehicle Age', 'Credit Score', 'Insurance Duration', 'Policy Start Date', 'Customer Feedback', 'Smoking Status', 'Exercise Frequency', 'Property Type']
cat_columns=['Gender',
 'Marital Status',
 'Education Level',
 'Occupation',
 'Location',
 'Policy Type',
 'Policy Start Date',
 'Customer Feedback',
 'Smoking Status',
 'Exercise Frequency',
 'Property Type']
num_columns=['Age',
 'Annual Income',
 'Number of Dependents',
 'Health Score',
 'Previous Claims',
 'Vehicle Age',
 'Credit Score',
 'Insurance Duration']


In [5]:
dataset_df_describe_num = eda_df(dataset_df[params["num_columns"]+[params["target_column"]]])
display(dataset_df_describe_num)

,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Premium Amount
count,1.181295e+06,1.155051e+06,1.090328e+06,1.125924e+06,835971.000000,1.199994e+06,1.062118e+06,1.199999e+06,1.200000e+06
mean,4.114556e+01,3.274522e+04,2.009934e+00,2.561391e+01,1.002689,9.569889e+00,5.929244e+02,5.018219e+00,1.102545e+03
std,1.353995e+01,3.217951e+04,1.417338e+00,1.220346e+01,0.982840,5.776189e+00,1.499819e+02,2.594331e+00,8.649989e+02
min,1.800000e+01,1.000000e+00,0.000000e+00,2.012237e+00,0.000000,0.000000e+00,3.000000e+02,1.000000e+00,2.000000e+01
10%,2.200000e+01,2.627000e+03,0.000000e+00,9.936849e+00,0.000000,2.000000e+00,3.830000e+02,1.000000e+00,1.500000e+02
25%,3.000000e+01,8.001000e+03,1.000000e+00,1.591896e+01,0.000000,5.000000e+00,4.680000e+02,3.000000e+00,5.140000e+02
50%,4.100000e+01,2.391100e+04,2.000000e+00,2.457865e+01,1.000000,1.000000e+01,5.950000e+02,5.000000e+00,8.720000e+02
75%,5.300000e+01,4.463400e+04,3.000000e+00,3.452721e+01,2.000000,1.500000e+01,7.210000e+02,7.000000e+00,1.509000e+03
90%,6.000000e+01,8.107500e+04,4.000000e+00,4.328134e+01,2.000000,1.800000e+01,7.920000e+02,9.000000e+00,2.363000e+03
max,6.400000e+01,1.499970e+05,4.000000e+00,5.897591e+01,9.000000,1.900000e+01,8.490000e+02,9.000000e+00,4.999000e+03


In [6]:
print("Уникальные значения по столбцам")
nunique = dataset_df[params["columns_X"]].nunique()
for column in nunique.index:
    if nunique[column] <= 10:
        print(f'{column:20}: {nunique[column]:6}, {dataset_df[column].unique().tolist()}')
    else:
        print(f'{column:20}: {nunique[column]:6}')


Уникальные значения по столбцам
Age                 :     47
Gender              :      2, ['Female', 'Male']
Annual Income       :  88593
Marital Status      :      3, ['Married', 'Divorced', 'Single', nan]
Number of Dependents:      5, [1.0, 3.0, 2.0, 0.0, 4.0, nan]
Education Level     :      4, ["Bachelor's", "Master's", 'High School', 'PhD']
Occupation          :      3, ['Self-Employed', nan, 'Employed', 'Unemployed']
Health Score        : 532657
Location            :      3, ['Urban', 'Rural', 'Suburban']
Policy Type         :      3, ['Premium', 'Comprehensive', 'Basic']
Previous Claims     :     10, [2.0, 1.0, 0.0, nan, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]
Vehicle Age         :     20
Credit Score        :    550
Insurance Duration  :      9, [5.0, 2.0, 3.0, 1.0, 4.0, 6.0, 8.0, 9.0, 7.0, nan]
Policy Start Date   : 167381
Customer Feedback   :      3, ['Poor', 'Average', 'Good', nan]
Smoking Status      :      2, ['No', 'Yes']
Exercise Frequency  :      4, ['Weekly', 'Monthly', 'D

In [7]:
dataset_df_describe_cat = dataset_df[params["cat_columns"]].describe()
display(dataset_df_describe_cat)

,Gender,Marital Status,Education Level,Occupation,Location,Policy Type,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type
count,1200000,1181471,1200000,841925,1200000,1200000,1200000,1122176,1200000,1200000,1200000
unique,2,3,4,3,3,3,167381,3,2,4,3
top,Male,Single,Master's,Employed,Suburban,Premium,2020-02-08 15:21:39.134960,Average,Yes,Weekly,House
freq,602571,395391,303818,282750,401542,401846,142,377905,601873,306179,400349


In [8]:
# сохранить промежуточные результаты

_ = joblib.dump(dataset_df, Path(result_foler, dataset_filename_after_EDA), compress=3)
_ = joblib.dump(params, Path(result_foler, params_filename_after_EDA), compress=3)